In [6]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
from PIL import Image

In [4]:
import torchvision
from torchvision import datasets, transforms, models

In [3]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F


In [2]:
import numpy as np

In [ ]:
# AI model to classify space rocks
import matplotlib.pyplot as plt